<a href="https://colab.research.google.com/github/DeekshaKarkada/Language-Translation-for-Hindi-Kannada/blob/main/mT5_Domain_Mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of language translation for Hindi to Kannada

This was implemented using mT5 model for OPUS dataset

In [ ]:
! pip install datasets sacrebleu transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import warnings
import numpy as np
import pandas as pd
import datasets
import torch
import transformers
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, MT5Tokenizer, MT5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_metric
from sacrebleu import corpus_bleu

warnings.filterwarnings("ignore")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Load the OPUS NLLB dataset containing data of wikipedia domain

In [ ]:
# load the Opus NLLB dataset
def load_data(file_name):
  with open(file_name, 'r', encoding='utf-8') as f:
    lines = f.readlines()
  return lines

src_lines = load_data('/content/drive/MyDrive/Thesis/NLLB_hi_kn-hi.txt')
tgt_lines = load_data('/content/drive/MyDrive/Thesis/NLLB_hi_kn-kn.txt')

dataset = pd.DataFrame({'src': src_lines, 'tgt': tgt_lines})

dataset = dataset.drop_duplicates().sample(frac=0.02).reset_index(drop=True)

In [ ]:
dataset.shape

(35878, 2)

# Load TED2020 dataset from TEDTalk domain

The dataset is available in https://opus.nlpl.eu/TED2020/hi&kn/v1/TED2020

In [ ]:
#Load the Opus TED2020 dataset
unseen_hi = load_data('/content/drive/MyDrive/Thesis/TED2020_hi-kn_hi.txt')
unseen_kn = load_data('/content/drive/MyDrive/Thesis/TED2020_hi-kn_kn.txt')

unseen_data = pd.DataFrame({'src': unseen_hi, 'tgt': unseen_kn})

In [ ]:
unseen_data.shape

(1214, 2)

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.25)
train_data, validation_data = train_test_split(train_data, test_size=0.35)

small_train, small_test = train_test_split(unseen_data, test_size=0.25)


# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(train_data)
validation_dataset = Dataset.from_pandas(validation_data)
test_dataset = Dataset.from_pandas(test_data)

small_train = Dataset.from_pandas(small_train)
small_test = Dataset.from_pandas(small_test)

In [ ]:
small_test.shape

(304, 3)

In [ ]:
# Initialize the tokenizer and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'google/mt5-base'
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [ ]:
# Process both the original and smaller dataset

def preprocess_function(examples):
    inputs = examples["src"]
    targets = examples["tgt"]

    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, padding="max_length")
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, desc="Running tokenizer on train dataset")
validation_dataset = validation_dataset.map(preprocess_function, batched=True, desc="Running tokenizer on validation dataset")
test_dataset = test_dataset.map(preprocess_function, batched=True, desc="Running tokenizer on test dataset")

small_train = small_train.map(preprocess_function, batched=True, desc="Running tokenizer on small train dataset")
small_test = small_test.map(preprocess_function, batched=True, desc="Running tokenizer on small test dataset")


Running tokenizer on train dataset:   0%|          | 0/17490 [00:00<?, ? examples/s]

Running tokenizer on validation dataset:   0%|          | 0/9418 [00:00<?, ? examples/s]

Running tokenizer on test dataset:   0%|          | 0/8970 [00:00<?, ? examples/s]

Running tokenizer on small train dataset:   0%|          | 0/910 [00:00<?, ? examples/s]

Running tokenizer on small test dataset:   0%|          | 0/304 [00:00<?, ? examples/s]

In [ ]:
train_dataset.shape

(17490, 6)

# Train the model for the train and validation set and get the evaluation on test data

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Save model at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=6,
    predict_with_generate=True,
    logging_dir='./logs',
    gradient_accumulation_steps=2,
    warmup_steps=500,
    eval_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the BLEU metric
bleu_metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_labels = [[label] for label in decoded_labels]
    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": bleu['score']}

# Initialize the trainer for the initial training phase
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


The repository for sacrebleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sacrebleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
trainer.train()

trainer.save_model("./base_large_model")


Epoch,Training Loss,Validation Loss,Bleu
0,0.580500,0.434335,2.902696
2,0.468000,0.397403,4.424737
4,0.437500,0.386839,4.874037
5,0.426300,0.385827,4.925697


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=26232, training_loss=1.0708866258989422, metrics={'train_runtime': 33152.1903, 'train_samples_per_second': 6.331, 'train_steps_per_second': 0.791, 'total_flos': 6.290675625413837e+16, 'train_loss': 1.0708866258989422, 'epoch': 5.999313893653516})

# Evaluation score and example translation

In [ ]:
import sacrebleu
# Generate translations and calculate BLEU score for test set
def generate_translation(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs).to(device)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Generate translations for the test set
refs = [[ex["tgt"]] for ex in test_dataset]
preds = [generate_translation(ex["src"]) for ex in test_dataset]

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(f"Test BLEU score after fine-tuning: {bleu.score}")

Test BLEU score after fine-tuning: 8.25791079503452


In [ ]:
input_text = "यह एक परीक्षण है।"
result = generate_translation(input_text)
print(result)


ಈ ಒಂದು ಸವಾಲು.
